In [3]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


ames = pd.read_csv('/Users/avanti/OneDrive/MASTERS/GSB544-Computing and Machine Learning/Practice Activities/PA_7.1/AmesHousing.csv')

# Get rid of columns with mostly NaN values
good_cols = ames.isna().sum() < 100
ames = ames.loc[:,good_cols]

# Drop other NAs
ames = ames.dropna()

ames.head()

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Garage Cars,Garage Area,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,2.0,528.0,P,210,62,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,1.0,730.0,Y,140,0,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,1.0,312.0,Y,393,36,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,2.0,522.0,Y,0,0,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,2.0,482.0,Y,212,34,0,0,0,0,0,3,2010,WD,Normal,189900


In [4]:
lr = LinearRegression()

X = ames[["Gr Liv Area", "TotRms AbvGrd"]]
y = ames["SalePrice"]



X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train_s = (X_train - X_train.mean())/X_train.std()

lr_fitted = lr.fit(X_train_s, y_train)
lr_fitted.coef_

array([ 70400.99187538, -17847.48237565])

In [5]:
y_preds = lr_fitted.predict(X_test)

r2_score(y_test, y_preds)

-1939974.1688547109

In [6]:
y_preds[1:5]

array([1.21991150e+08, 1.55043921e+08, 1.28608843e+08, 8.11238685e+07])

In [7]:
new_house = pd.DataFrame(data = {"Gr Liv Area": [889], "TotRms AbvGrd": [6]})
new_house

,Gr Liv Area,TotRms AbvGrd
0,889,6


In [8]:
new_house_s = (new_house - new_house.mean())/new_house.std()
new_house_s

,Gr Liv Area,TotRms AbvGrd
0,NaN,NaN


In [9]:
X_test_s = (X_test - X_train.mean())/X_train.std()
y_preds = lr_fitted.predict(X_test_s)

r2_score(y_test, y_preds)

0.5132137299431012

In [10]:
new_house_s = (new_house - X_train.mean())/X_train.std()
lr_fitted.predict(new_house_s)

array([99211.46944644])

In [11]:
lr_pipeline = Pipeline(
  [("standardize", StandardScaler()),
  ("linear_regression", LinearRegression())]
)

lr_pipeline

Pipeline(steps=[('standardize', StandardScaler()),
                ('linear_regression', LinearRegression())])

In [12]:
lr_pipeline_fitted = lr_pipeline.fit(X_train, y_train)

y_preds = lr_pipeline_fitted.predict(X_test)
r2_score(y_test, y_preds)

0.5132137299431013

In [13]:
lr_pipeline_fitted.predict(new_house)

array([99211.46944644])

In [14]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)


lr_pipeline = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

lr_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Bldg Type']),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

In [15]:
X = ames.drop("SalePrice", axis = 1)
y = ames["SalePrice"]



X_train, X_test, y_train, y_test = train_test_split(X, y)

lr_fitted = lr_pipeline.fit(X_train, y_train)

In [16]:
ct_fitted = ct.fit(X_train)

ct.transform(X_train)

array([[ 1.        ,  0.        ,  0.        , ...,  0.        , -0.63916584,
        -0.26681461],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,  0.5230414 ,
         0.36867289],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,  0.8071365 ,
         0.36867289],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        , -1.58482506,
        -1.53778963],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,  0.41377405,
        -0.26681461],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,  0.08994537,
         0.36867289]])

In [17]:
ct.transform(X_test)

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.00000000e+00, -1.72296266e-01, -2.66814613e-01],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  5.44810327e-04, -2.66814613e-01],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -5.61685357e-01, -2.66814613e-01],
       ...,
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  1.31665627e-01, -9.02302121e-01],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  6.72042325e-01,  3.68672895e-01],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -3.51097379e-01, -9.02302121e-01]])

In [18]:
lr_pipeline_fitted.named_steps['linear_regression'].coef_

array([ 70384.34664714, -17843.26261383])

In [19]:
type(y_preds)

numpy.ndarray

In [20]:
lr_pipeline = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
).set_output(transform="pandas")


ct.fit_transform(X_train)

,dummify__Bldg Type_1Fam,dummify__Bldg Type_2fmCon,dummify__Bldg Type_Duplex,dummify__Bldg Type_Twnhs,dummify__Bldg Type_TwnhsE,standardize__Gr Liv Area,standardize__TotRms AbvGrd
1964,1.0,0.0,0.0,0.0,0.0,-0.639166,-0.266815
2215,1.0,0.0,0.0,0.0,0.0,0.523041,0.368673
1540,1.0,0.0,0.0,0.0,0.0,0.807136,0.368673
960,1.0,0.0,0.0,0.0,0.0,0.296560,0.368673
2606,1.0,0.0,0.0,0.0,0.0,-0.333217,-0.266815
...,...,...,...,...,...,...,...
2562,1.0,0.0,0.0,0.0,0.0,0.171399,-0.902302
1834,0.0,0.0,0.0,0.0,1.0,-0.225937,-0.266815
2206,1.0,0.0,0.0,0.0,0.0,-1.584825,-1.537790
141,1.0,0.0,0.0,0.0,0.0,0.413774,-0.266815


In [21]:
ct_inter = ColumnTransformer(
  [
    ("interaction", PolynomialFeatures(interaction_only = True), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
).set_output(transform = "pandas")

ct_inter.fit_transform(X_train)

,interaction__1,interaction__Gr Liv Area,interaction__TotRms AbvGrd,interaction__Gr Liv Area TotRms AbvGrd
1964,1.0,1170.0,6.0,7020.0
2215,1.0,1755.0,7.0,12285.0
1540,1.0,1898.0,7.0,13286.0
960,1.0,1641.0,7.0,11487.0
2606,1.0,1324.0,6.0,7944.0
...,...,...,...,...
2562,1.0,1578.0,5.0,7890.0
1834,1.0,1378.0,6.0,8268.0
2206,1.0,694.0,4.0,2776.0
141,1.0,1700.0,6.0,10200.0


In [22]:
ct_dummies = ColumnTransformer(
  [("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])],
  remainder = "passthrough"
).set_output(transform = "pandas")

ct_inter = ColumnTransformer(
  [
    ("interaction", PolynomialFeatures(interaction_only = True), ["remainder__TotRms AbvGrd", "dummify__Bldg Type_1Fam"]),
  ],
  remainder = "drop"
).set_output(transform = "pandas")

X_train_dummified = ct_dummies.fit_transform(X_train)
X_train_dummified

,dummify__Bldg Type_1Fam,dummify__Bldg Type_2fmCon,dummify__Bldg Type_Duplex,dummify__Bldg Type_Twnhs,dummify__Bldg Type_TwnhsE,remainder__Order,remainder__PID,remainder__MS SubClass,remainder__MS Zoning,remainder__Lot Area,remainder__Street,remainder__Lot Shape,remainder__Land Contour,remainder__Utilities,remainder__Lot Config,remainder__Land Slope,remainder__Neighborhood,remainder__Condition 1,remainder__Condition 2,remainder__House Style,remainder__Overall Qual,remainder__Overall Cond,remainder__Year Built,remainder__Year Remod/Add,remainder__Roof Style,remainder__Roof Matl,remainder__Exterior 1st,remainder__Exterior 2nd,remainder__Mas Vnr Area,remainder__Exter Qual,remainder__Exter Cond,remainder__Foundation,remainder__Bsmt Qual,remainder__Bsmt Cond,remainder__Bsmt Exposure,remainder__BsmtFin Type 1,remainder__BsmtFin SF 1,remainder__BsmtFin Type 2,remainder__BsmtFin SF 2,remainder__Bsmt Unf SF,remainder__Total Bsmt SF,remainder__Heating,remainder__Heating QC,remainder__Central Air,remainder__Electrical,remainder__1st Flr SF,remainder__2nd Flr SF,remainder__Low Qual Fin SF,remainder__Gr Liv Area,remainder__Bsmt Full Bath,remainder__Bsmt Half Bath,remainder__Full Bath,remainder__Half Bath,remainder__Bedroom AbvGr,remainder__Kitchen AbvGr,remainder__Kitchen Qual,remainder__TotRms AbvGrd,remainder__Functional,remainder__Fireplaces,remainder__Garage Cars,remainder__Garage Area,remainder__Paved Drive,remainder__Wood Deck SF,remainder__Open Porch SF,remainder__Enclosed Porch,remainder__3Ssn Porch,remainder__Screen Porch,remainder__Pool Area,remainder__Misc Val,remainder__Mo Sold,remainder__Yr Sold,remainder__Sale Type,remainder__Sale Condition
1964,1.0,0.0,0.0,0.0,0.0,1965,535453150,20,RL,7315,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Story,5,6,1958,1958,Hip,CompShg,BrkFace,BrkFace,0.0,TA,TA,CBlock,TA,TA,No,Rec,625.0,Unf,0.0,545.0,1170.0,GasA,TA,Y,SBrkr,1170,0,0,1170,0.0,1.0,1,0,3,1,TA,6,Typ,1,1.0,338.0,Y,0,0,0,0,225,0,0,3,2007,WD,Normal
2215,1.0,0.0,0.0,0.0,0.0,2216,909276210,50,RL,7810,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1.5Fin,4,6,1930,2003,Gable,CompShg,AsbShng,CmentBd,0.0,TA,Gd,BrkTil,TA,Gd,No,GLQ,189.0,Unf,0.0,741.0,930.0,GasA,Ex,Y,SBrkr,1230,525,0,1755,0.0,0.0,2,0,4,1,Gd,7,Typ,1,1.0,231.0,Y,0,40,0,0,0,0,0,11,2007,WD,Normal
1540,1.0,0.0,0.0,0.0,0.0,1541,909428170,20,RL,26073,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Crawfor,Norm,Norm,1Story,5,5,1956,1956,Gable,CompShg,BrkFace,MetalSd,0.0,TA,TA,PConc,TA,TA,No,Rec,1116.0,Unf,0.0,782.0,1898.0,GasA,Ex,Y,FuseA,1898,0,0,1898,0.0,0.0,2,1,3,1,TA,7,Typ,2,2.0,484.0,Y,0,51,224,0,0,0,0,4,2008,WD,Normal
960,1.0,0.0,0.0,0.0,0.0,961,916382110,20,RL,14463,Pave,IR1,HLS,AllPub,Corner,Gtl,Timber,Norm,Norm,1Story,9,5,2008,2008,Gable,CompShg,CemntBd,CmentBd,406.0,Ex,TA,PConc,Ex,TA,Gd,GLQ,1115.0,Unf,0.0,526.0,1641.0,GasA,Ex,Y,SBrkr,1641,0,0,1641,1.0,0.0,2,0,3,1,Ex,7,Typ,0,3.0,885.0,Y,0,95,0,0,0,0,0,1,2009,WD,Normal
2606,1.0,0.0,0.0,0.0,0.0,2607,535401140,20,RL,8658,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Story,6,5,1965,1965,Gable,CompShg,Wd Sdng,Wd Sdng,101.0,TA,TA,CBlock,TA,TA,No,Rec,643.0,Unf,0.0,445.0,1088.0,GasA,Ex,Y,SBrkr,1324,0,0,1324,0.0,0.0,2,0,3,1,TA,6,Typ,1,2.0,440.0,Y,0,138,0,0,0,0,0,12,2006,WD,Abnorml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562,1.0,0.0,0.0,0.0,0.0,2563,534479130,50,RL,9490,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Artery,Norm,1.5Fin,6,7,1941,1950,Gable,CompShg,Wd Sdng,Wd Sdng,0.0,TA,TA,CBlock,TA,TA,No,BLQ,403.0,Rec,165.0,238.0,806.0,GasA,TA,Y,FuseA,958,620,0,1578,1.0,0.0,1,0,3,1,Fa,5,Typ,2,1.0,240.0,Y,0,0,32,0,0,0,0,8,2006,WD,Normal
1834,0.0,0.0,0.0,0.0,1.0,1835,533205110,120,FV,4379,Pave,IR1,Lvl,AllPub,FR2,Gtl,Somerst,Norm,Norm,1Story,8,5,2004,2004,Gable,CompShg,MetalSd,MetalSd,0.0,Gd,TA,PConc,Gd,TA,A

In [23]:
ct_inter.fit_transform(X_train_dummified)

,interaction__1,interaction__remainder__TotRms AbvGrd,interaction__dummify__Bldg Type_1Fam,interaction__remainder__TotRms AbvGrd dummify__Bldg Type_1Fam
1964,1.0,6.0,1.0,6.0
2215,1.0,7.0,1.0,7.0
1540,1.0,7.0,1.0,7.0
960,1.0,7.0,1.0,7.0
2606,1.0,6.0,1.0,6.0
...,...,...,...,...
2562,1.0,5.0,1.0,5.0
1834,1.0,6.0,0.0,0.0
2206,1.0,4.0,1.0,4.0
141,1.0,6.0,1.0,6.0


Consider four possible models for predicting house prices:

Using only the size and number of rooms.
Using size, number of rooms, and building type.
Using size and building type, and their interaction.
Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.
Set up a pipeline for each of these four models.

Then, get predictions on the test set for each of your pipelines, and compute the root mean squared error. Which model performed best?

Note: You should only use the function train_test_split() one time in your code; that is, we should be predicting on the same test set for all three models.

In [25]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Preprocess
good_cols = ames.isna().sum() < 100
ames = ames.loc[:, good_cols].dropna()

# Define target and feature columns
target = "SalePrice"
features = ["Gr Liv Area", "TotRms AbvGrd", "Bldg Type"]

X = ames[features]
y = ames[target]

# Perform train/test split once
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Model 1: Using only size and number of rooms
pipeline1 = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_regression", LinearRegression())
])

# Model 2: Using size, number of rooms, and building type
pipeline2 = Pipeline([
    ("preprocessor", ColumnTransformer([
        ("scale_num", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"]),
        ("encode_cat", OneHotEncoder(drop="first"), ["Bldg Type"])
    ])),
    ("linear_regression", LinearRegression())
])

# Model 3: Using size and building type, and their interaction
pipeline3 = Pipeline([
    ("preprocessor", ColumnTransformer([
        ("scale_num", StandardScaler(), ["Gr Liv Area"]),
        ("encode_cat", OneHotEncoder(drop="first"), ["Bldg Type"]),
        ("interaction", PolynomialFeatures(interaction_only=True), ["Gr Liv Area"])
    ])),
    ("linear_regression", LinearRegression())
])

# Model 4: Using 5-degree polynomial on size and number of rooms, and building type
pipeline4 = Pipeline([
    ("preprocessor", ColumnTransformer([
        ("poly_features", PolynomialFeatures(degree=5), ["Gr Liv Area", "TotRms AbvGrd"]),
        ("encode_cat", OneHotEncoder(drop="first"), ["Bldg Type"])
    ])),
    ("linear_regression", LinearRegression())
])

# Fit each pipeline and evaluate
pipelines = [pipeline1, pipeline2, pipeline3, pipeline4]
rmse_results = {}

for i, pipeline in enumerate(pipelines, start=1):
    # Fit the pipeline
    pipeline.fit(X_train, y_train)
    # Predict on the test set
    y_pred = pipeline.predict(X_test)
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_results[f"Model {i}"] = rmse

# Display RMSE results
rmse_results

ValueError: could not convert string to float: '1Fam'